In [2]:
import os
import time
import json
import requests
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Dict

load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

def fetch_naver_blog(keyword: str, display: int = 50) -> List[Dict]:
    url = "https://openapi.naver.com/v1/search/blog.json"
    params = {
        "query": keyword,
        "display": display,
        "start": 1,
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    items = response.json().get("items", [])

    result = []
    for item in items:
        result.append({
            "type": "blog",
            "title": item.get("title"),
            "url": item.get("link"),
            "keyword": keyword,
            "summary": item.get("description"),
            "image_url": None,
            "published_at": datetime.strptime(item["postdate"], "%Y%m%d").isoformat() if "postdate" in item else None,
            "created_at": datetime.now().isoformat(),
            "author": item.get("bloggername"),
            "sentiment_score": None,
            "word_count": None
        })

    return result

In [3]:
# 종목 키워드 리스트
keywords = [
    "삼성전자", "SK하이닉스", "네이버", "카카오", "삼성바이오로직스",
    "LG화학", "현대차", "기아", "LG에너지솔루션", "POSCO홀딩스",
    "현대모비스", "셀트리온", "카카오게임즈", "크래프톤", "HMM",
    "대한항공", "DB하이텍", "아모레퍼시픽", "롯데쇼핑", "CJ ENM"
]

# 블로그 수집 및 저장
all_blogs = []
for kw in keywords:
    try:
        blogs = fetch_naver_blog(kw, display=50)
        all_blogs.extend(blogs)
        time.sleep(1)
    except Exception as e:
        print(f"Error processing {kw}: {e}")

# 저장
output_dir = "./data"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "naver_blog_results.json"), "w", encoding="utf-8") as f:
    json.dump(all_blogs, f, ensure_ascii=False, indent=2)

output_path

NameError: name 'output_path' is not defined